In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

<p>ChromeDriver is a separate executable that Selenium WebDriver uses to control Chrome. It is maintained by the Chromium team with help from WebDriver contributors. If you are unfamiliar with Selenium WebDriver, you should check out the <a href="https://www.selenium.dev/">Selenium site</a>.</p>

https://googlechromelabs.github.io/chrome-for-testing/

# 1. Create df_state_universities dataframe.
#### Index of the dataframe is university code and the single column is the university name.

In [15]:
service = Service("chromedriver.exe")
driver= webdriver.Chrome(service=service)
driver.get("https://yokatlas.yok.gov.tr/lisans-anasayfa.php")
driver.maximize_window()
driver.implicitly_wait(5)
optgroups = driver.find_elements(by=By.XPATH,value="//optgroup[@label]")[:4]
labels = [optgroup.get_attribute("label").split()[0]   for optgroup in optgroups]
universities = { }
 
for label , optgroup in zip(labels,optgroups):
    options = optgroup.find_elements (by=By.TAG_NAME,value="option")
    universities[label] = {int(option.get_attribute("value")):option.get_attribute("innerText").strip()  for option in options}
df_universities =pd.DataFrame.from_dict(universities).stack().to_frame().reset_index()
df_universities.columns = ["uni_code","type","name"]
df_universities.head(3)  

,uni_code,type,name
0,1065,Devlet,ABDULLAH GÜL ÜNİVERSİTESİ
1,1104,Devlet,ADANA ALPARSLAN TÜRKEŞ BİLİM VE TEKNOLOJİ ÜNİV...
2,1002,Devlet,ADIYAMAN ÜNİVERSİTESİ


# 2- Add the city column
#### Create new data frame with columns city name and university name

In [ ]:
driver.get("https://yokatlas.yok.gov.tr/universite.php")
driver.implicitly_wait(10)
city_web_elements = driver.find_elements(by=By.CLASS_NAME, value="sehir")

cities = [web_element.get_attribute("innerText").strip() for web_element in city_web_elements]

name_web_elements = driver.find_elements(by=By.CLASS_NAME, value="baslik")
names =[web_element.get_attribute("innerText").strip() for web_element in name_web_elements]
df_city = pd.DataFrame({"city":cities,"name":names})
df_city.head(3)

Merge two dataframes: df_state_universities and df_city

In [ ]:
#df_universities=df_universities.merge(df_city, on="name", how='left').set_index("uni_code")
#df_universities.head(3)

In [ ]:
df_universities.shape

In [ ]:
df_universities.set_index([df_universities.columns[1],df_universities.columns[0] ],inplace=True) 
df_universities.head(3)  

In [ ]:
df_universities.to_csv("df_universities.csv")

# 3- Create df_quota 
### df_quota includes all departments and their quota/enrollments 

In [29]:
df_universities = pd.read_csv("df_universities.csv",index_col=0)
df_universities.head()

,type,name,city
uni_code,,,
1065,Devlet,ABDULLAH GÜL ÜNİVERSİTESİ,Kayseri
1104,Devlet,ADANA ALPARSLAN TÜRKEŞ BİLİM VE TEKNOLOJİ ÜNİV...,Adana
1002,Devlet,ADIYAMAN ÜNİVERSİTESİ,Adıyaman
1004,Devlet,AFYON KOCATEPE ÜNİVERSİTESİ,Afyonkarahisar
1126,Devlet,AFYONKARAHİSAR SAĞLIK BİLİMLERİ ÜNİVERSİTESİ,Afyonkarahisar


In [1]:
base_url = "https://yokatlas.yok.gov.tr/lisans-univ.php?u="

In [17]:
def get_department_names(driver,base_url, uni_codes):
    department_names = set()
    for uni_code in uni_codes:
        driver.get(base_url + str(uni_code) )  
        driver.implicitly_wait(2)
        divs_departments =  driver.find_elements(by=By.XPATH, value="//a[@data-parent='#']/div")
        for div_department  in divs_departments:
            department_name = div_department.get_attribute("innerText")
            if "KKTC" not in department_name:
                department_names.add(department_name) 
                
    return department_names    

In [18]:
 df_universities.head(3)

,uni_code,type,name
0,1065,Devlet,ABDULLAH GÜL ÜNİVERSİTESİ
1,1104,Devlet,ADANA ALPARSLAN TÜRKEŞ BİLİM VE TEKNOLOJİ ÜNİV...
2,1002,Devlet,ADIYAMAN ÜNİVERSİTESİ


In [19]:
department_names = get_department_names(driver,base_url, df_universities["uni_code"])
len(department_names)

1545

In [35]:
department_names = list(department_names)
department_names.sort()
midx = pd.MultiIndex.from_product([ department_names, ["quota","enrollments"]  ]) 
df_quota = pd.DataFrame(index = midx,columns=df_universities.index)
df_quota.head()

NameError: name 'department_names' is not defined

In [25]:
def get_row_content(driver,searched_text):
    x_path = "//tr[contains(.,"+searched_text +")]"
    td = WebDriverWait(driver, 3).until( EC.presence_of_element_located((By.XPATH, x_path)) )  
    # Find the td element within the row_element
    tr = td.find_element(By.XPATH, './td[last()]')  # Selecting the last td element
    return tr.get_attribute("innerText")

In [62]:
wait_time = .2
imp_wait_time= .2
def get_quota_enrolled(driver):
    # close pop-ups
    try:
        pop_up_element_xpath = "//span[@class='featherlight-close-icon featherlight-close']"
        WebDriverWait(driver,wait_time).until( EC.presence_of_element_located((By.XPATH,pop_up_element_xpath) )  )
        #driver.implicitly_wait(5)
        pop_ups = driver.find_elements(by=By.XPATH, value=pop_up_element_xpath)
        for i in range(len(pop_ups)-1,-1,-1): 
            pop_ups[i].click() 
    except:
        pass
    
    result=False
    while not result:
        try:
            WebDriverWait(driver, wait_time).until( EC.presence_of_element_located((By.XPATH,"//*[contains(text(),'Genel Bilgiler')]") )  ).click()
            quota = get_row_content(driver,searched_text= "'Toplam Kontenjan'")         
            enrollments = get_row_content(driver,searched_text= "'Toplam Yerleşen'")
   
            result=True
        except:
            pass
        
    
    return quota, enrollments

In [64]:
def get_department_anchors_divs(driver):
  #  time.sleep(2)
    driver.implicitly_wait(imp_wait_time)
    anchors          =  driver.find_elements(by=By.XPATH, value="//a[@data-parent='#']") #links to departments
   # time.sleep(2)
    driver.implicitly_wait(imp_wait_time)
    divs_departments =  driver.find_elements(by=By.XPATH, value="//a[@data-parent='#']/div")
    return anchors, divs_departments

In [65]:
def create_df_quota(driver, df_quota,uni_code,sleep_time=2):
    
    driver.implicitly_wait(imp_wait_time)
    driver.get(base_url + str(uni_code) )
   # time.sleep(sleep_time)   
    driver.implicitly_wait(imp_wait_time)
    anchors, divs_departments = get_department_anchors_divs(driver)
  #  time.sleep(sleep_time)  
    for anchor,div_department in zip(anchors, divs_departments):
        result=False
        while not result:
          #  time.sleep(sleep_time)
            try:           
                driver.implicitly_wait(imp_wait_time)
                department_name = div_department.get_attribute("innerText").strip()  
                result=True
            except:
                print("Failed to get department name. uni_code:",uni_code)
                exit_result = create_df_quota(driver, df_quota,uni_code, 5) #recursive call to restart scraping the same page
                if exit_result:
                    return True
        if "KKTC" not in department_name:         
            driver.get(anchor.get_attribute("href"))
            driver.implicitly_wait(imp_wait_time)  
            quota, enrollments =  get_quota_enrolled(driver)           
            df_quota.loc[department_name,"quota"][uni_code] = int(quota)
            df_quota.loc[department_name,"enrollments"][uni_code] = int(enrollments) 
            print(department_name,"int(quota)",int(quota)," int(enrollments) ", int(enrollments) )    
            driver.implicitly_wait(imp_wait_time)
            driver.back()    
    print("Completed "+str(uni_code),end=",")
    return True

In [66]:
for uni_code in df_universities.index[:1]: #list(uni_codes):#
    create_df_quota(driver, df_quota,uni_code)    

Bilgisayar Mühendisliği (İngilizce) int(quota) 77  int(enrollments)  77
Biyomühendislik (İngilizce) int(quota) 67  int(enrollments)  67
Ekonomi (İngilizce) int(quota) 54  int(enrollments)  52
Elektrik-Elektronik Mühendisliği (İngilizce) int(quota) 88  int(enrollments)  88
Endüstri Mühendisliği (İngilizce) int(quota) 72  int(enrollments)  72
İnşaat Mühendisliği (İngilizce) int(quota) 72  int(enrollments)  72
İşletme (İngilizce) int(quota) 64  int(enrollments)  62
Makine Mühendisliği (İngilizce) int(quota) 62  int(enrollments)  62
Mimarlık (İngilizce) int(quota) 72  int(enrollments)  72
Moleküler Biyoloji ve Genetik (İngilizce) int(quota) 64  int(enrollments)  62
Psikoloji (İngilizce) int(quota) 64  int(enrollments)  64
Siyaset Bilimi ve Uluslararası İlişkiler (İngilizce) int(quota) 64  int(enrollments)  64
Completed 1065,

In [ ]:
df_quota.info()

In [ ]:
df_quota.to_csv("df_quota.csv")

In [4]:
service = Service("chromedriver.exe")
driver= webdriver.Chrome(service=service)
driver.maximize_window()

In [52]:
df_quota = pd.read_csv("df_quota.csv",index_col=[0,1])

In [55]:
df_quota.columns=[int(c) for c in df_quota.columns]

In [48]:
df_quota.loc['Acil Yardım ve Afet Yönetimi','quota']["1065"]

nan

In [56]:
df_quota.columns[0]

1065